In [7]:
import os
import torch
import glob
from monai.networks.nets import AttentionUnet
from monai.inferers import SlidingWindowInferer
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd, Orientationd,
    ScaleIntensityRanged, CropForegroundd, DivisiblePadd, EnsureTyped,
    Activationsd, AsDiscreted, Invertd, SaveImaged
)
from monai.data import Dataset, DataLoader, decollate_batch

In [8]:
def run_inference_many_files_one_by_one():
    model_path = "/mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/saved_models/best_model.pth"
    input_folder = "/mrhung_nguyen_minh_quang_108/workspace/train/nnUNet_raw/Dataset015_lungTumor/imagesTs"
    output_folder = "/mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference_many"

    os.makedirs(output_folder, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Sử dụng thiết bị: {device}")

    model = AttentionUnet(
        spatial_dims=3,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
    ).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Transforms
    pre_transforms = Compose([
        LoadImaged(keys="image"),
        EnsureChannelFirstd(keys="image"),
        Spacingd(keys="image", pixdim=(2.0, 2.0, 2.0), mode="bilinear"),
        Orientationd(keys="image", axcodes="RAS"),
        ScaleIntensityRanged(keys="image", a_min=-1000, a_max=400, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys="image", source_key="image", allow_smaller=True),
        DivisiblePadd(keys="image", k=16),
        EnsureTyped(keys="image"),
    ])

    post_transforms = Compose([
        Activationsd(keys="pred", sigmoid=True),
        Invertd(
            keys="pred",
            transform=pre_transforms,
            orig_keys="image",
            nearest_interp=False,
            to_tensor=True,
        ),
        AsDiscreted(keys="pred", threshold=0.5),
        SaveImaged(keys="pred", output_dir=output_folder, output_postfix="atten", resample=False),
    ])

    inferer = SlidingWindowInferer(
        roi_size=[64, 64, 64],
        sw_batch_size=1,
        overlap=0.5
    )

    # Get list of files
    input_files = sorted(glob.glob(os.path.join(input_folder, "*.nii.gz")))
    print(f"Tìm thấy {len(input_files)} file cần xử lý...")

    for idx, file_path in enumerate(input_files):
        print(f"Đang xử lý file {idx+1}/{len(input_files)}: {os.path.basename(file_path)}")
        data_dict = [{"image": file_path}]
        dataset = Dataset(data=data_dict, transform=pre_transforms)
        loader = DataLoader(dataset, batch_size=1, num_workers=0)

        with torch.no_grad():
            for batch in loader:
                inputs = batch["image"].to(device)
                with torch.cuda.amp.autocast():
                    outputs = inferer(inputs, model)
                batch["pred"] = outputs
                results = [post_transforms(i) for i in decollate_batch(batch)]

    print(f"Hoàn tất inference cho {len(input_files)} file. Kết quả lưu tại: {output_folder}")


In [9]:
if __name__ == "__main__":
    run_inference_many_files_one_by_one()

Sử dụng thiết bị: cuda
Tìm thấy 29 file cần xử lý...
Đang xử lý file 1/29: lung_001_0000.nii.gz


/tmp/ipykernel_13411/3776864680.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
/tmp/ipykernel_13411/

2025-05-01 17:24:37,363 INFO image_writer.py:197 - writing: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference_many/lung_001_0000/lung_001_0000_atten.nii.gz
Đang xử lý file 2/29: lung_007_0000.nii.gz
2025-05-01 17:24:52,412 INFO image_writer.py:197 - writing: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference_many/lung_007_0000/lung_007_0000_atten.nii.gz
Đang xử lý file 3/29: lung_008_0000.nii.gz
2025-05-01 17:25:02,052 INFO image_writer.py:197 - writing: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference_many/lung_008_0000/lung_008_0000_atten.nii.gz
Đang xử lý file 4/29: lung_010_0000.nii.gz
2025-05-01 17:25:12,895 INFO image_writer.py:197 - writing: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference_many/lung_010_0000/lung_010_0000_atten.nii.gz
Đang xử lý file 5/29: lung_024_0000.nii.gz


RuntimeError: applying transform <monai.transforms.post.dictionary.Invertd object at 0x7fd87c859b10>